In [ ]:
# !pip install tensorflow

import tensorflow as tf
import numpy as np
import PIL
import os
import tensorflow_datasets as tdf

print(tf.__version__)

import pathlib
dataset_url= "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

roses = list(data_dir.glob('sunflowers/*'))
PIL.Image.open(str(roses[242]))

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = 5
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# model.compile(
#   train_ds,
#   validation_data=val_ds,
#   epochs=3
# # )
model.compile(
    optimizer='adam',  # Choose an optimizer, e.g., 'adam'
    loss='sparse_categorical_crossentropy',  # Choose a loss function
    metrics=['accuracy']  # Choose evaluation metrics
)

# Then, train the model using model.fit with validation_data and epochs
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

from google.colab import files

from PIL import Image # import the Image module from PIL

uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Load and preprocess the image
image = Image.open(image_path).resize((img_width, img_height)) # Now Image is defined
image_array = np.array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Make a prediction
predictions = model.predict(image_array)
predicted_class = class_names[np.argmax(predictions[0])]
print(f"The predicted class is: {predicted_class}")

# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import files
from PIL import Image
import numpy as np
import pathlib

# Load and preprocess the dataset (if not done already)
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Store class_names before applying map
class_names = train_ds.class_names # Store class names here


# Normalize the dataset
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

#class_names = train_ds.class_names  <-- Remove this line, it's causing the error.
print(class_names)

# Define the model
num_classes = len(class_names)  # Use the stored class_names here.
model = models.Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=2)

# Upload an image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Load and preprocess the image
image = Image.open(image_path).resize((img_width, img_height))
image_array = np.array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Make a prediction
predictions = model.predict(image_array)
predicted_class = class_names[np.argmax(predictions)] # Use the stored class_names
print(f"The predicted class is: {predicted_class}")

# Upload an image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Load and preprocess the image
image = Image.open(image_path).resize((img_width, img_height))
image_array = np.array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Make a prediction
predictions = model.predict(image_array)
predicted_class = class_names[np.argmax(predictions)] # Use the stored class_names
print(f"The predicted class is: {predicted_class}")



2.17.1
3670
Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
(32, 180, 180, 3)
(32,)
0.0 0.96902645
Epoch 1/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.1656 - loss: 8.9060 - val_accuracy: 0.1635 - val_loss: 8.4110
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.1684 - loss: 8.8562 - val_accuracy: 0.1635 - val_loss: 8.4110
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.1684 - loss: 8.8562 - val_accuracy: 0.1635 - val_loss: 8.4110


Saving sunflower1.jpg to sunflower1 (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
The predicted class is: roses
Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
Epoch 1/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.3881 - loss: 1.4237 - val_accuracy: 0.5913 - val_loss: 1.0615
Epoch 2/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.6180 - loss: 0.9795 - val_accuracy: 0.6090 - val_loss: 0.9772


Saving flower.jpg to flower.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
The predicted class is: roses


Saving flower2.jpg to flower2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
The predicted class is: sunflowers
